# Pointcloud Demo
This demonstration notebook uses code from https://github.com/heliophysicsPy/summer-school-24/tree/main/pysat-tutorial.

In [1]:
# import packages
import pandas as pd
import pysat
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from spacepy import coordinates as coord
import warnings
from harkness.create_ply import create_ply
warnings.filterwarnings("ignore", message="'S' is deprecated and will be removed in a future version, please use 's' instead.")

pd.set_option('display.max_columns', None)

/home/pliny/.local/lib/python3.9/site-packages/spacepy/time.py:2367: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


## Download data
As an example, we'll visualize ICON data during the 2020 Tonga eruption. First, let's download the data we're interested in using pysat and put it into pandas dataframes. 

In [2]:
# Set range of dates for downloading ICON data
# start_download_date = dt.datetime(2020, 12, 19)
# stop_download_date = dt.datetime(2020, 12, 24)
start_download_date = dt.datetime(2022, 1, 13)
stop_download_date = dt.datetime(2022, 1, 17)

In [3]:
# Set data directory if user hasn't already set one
print(f"old: {pysat.params['data_dirs']}")
if len(pysat.params['data_dirs']) == 0 or pysat.params['data_dirs'] == ['.']:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = 'output/'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

print(f"new: {pysat.params['data_dirs']}")

old: ['output']
pysat directory has been set previously. Leaving unchanged.
new: ['output']


### IVM
We want to see the following properties: 
```
'IVM': 'Altitude', 'Latitude', 'Longitude',
                     'Ion_Velocity_Field_Aligned',
                     'Ion_Velocity_Meridional', 'Ion_Velocity_Zonal',
                     'Fractional_Ion_Density_H', 'Fractional_Ion_Density_O',
                     'Ion_Density', 'Ion_Temperature']
```




In [4]:
# # # Instantiate IVM Instrument object. IVM data is automatically cleaned using
# # # instrument flags as it is loaded. Levels of 'clean', 'dusty', 'dirty', and
# # # 'none' or None are supported. This is generally true for all pysat instruments
# ivm = pysat.Instrument('icon', 'ivm', inst_id='a', clean_level='clean')

# # # Download data from NASA CDAWeb.

# # # If you would like additional feedback while pysat performs operations
# # # uncomment the line below.
# # pysat.logger.setLevel("INFO")

# # # Download data between specific dates. We shouldn't need to do this today
# ivm.download(start_download_date, stop_download_date)

# # # To get all of the latest files on the server but not on
# # # the local machine, uncomment line below.
# # # The version and revision numbers in the filenames are used to identify
# # # the latest files. This works for all ICON instruments.
# # # ivm.download_updated_files()

# # # Resume limited feedback
# pysat.logger.setLevel("WARNING")

# ivm.load(date = start_download_date, end_date = stop_download_date)

In [5]:
# help(pysat.Instrument)

### MIGHTI 
We want to see the following parameters:
```
'MIGHTI': 'Epoch', 'Altitude', 'Latitude', 'Longitude',
                    'Magnetic_Field_Aligned_Wind',
                    'Magnetic_Meridional_Wind', 'Magnetic_Zonal_Wind',
                    'Meridional_Wind', 'Meridional_Wind_Error',
                    'Zonal_Wind', 'Zonal_Wind_Error'
```

In [6]:
# # Register instruments with pysat. Only needed once per install.
# import pysatNASA
# pysat.utils.registry.register(['pysatNASA.instruments.icon_mighti'])

# # Improvements for loading ICON metadata are currently in
# # https://github.com/pysat/pysatNASA/pull/100.
# warnings.simplefilter('ignore', UserWarning)

# # First, obtain MIGHTI data from NASA CDAWeb.

# # Instantiate pysat.Instrument objects for the MIGHTI data products
# # MIGHTI Vector wind red.
# mighti_vw_red = pysat.Instrument('icon', 'mighti', tag='vector_wind_red',
#                                  inst_id='vector')

# # MIGHTI Vector wind green.
# mighti_vw_green = pysat.Instrument('icon', 'mighti', tag='vector_wind_green',
#                                    inst_id='vector')

# # MIGHTI Temperature.
# mighti_temp_a = pysat.Instrument('icon', 'mighti', tag='temperature',
#                                  inst_id='a')
# mighti_temp_b = pysat.Instrument('icon', 'mighti', tag='temperature',
#                                  inst_id='b')

# # Collect into a list.
# mighti_insts = [mighti_vw_red, mighti_vw_green, mighti_temp_a, mighti_temp_b]

In [7]:
# # Download various MIGHTI data products

# # If needed, change levels for logging printout to increase feedback.
# # More information about logging may be found here:
# # https://docs.python.org/3/library/logging.html
# pysat.logger.setLevel("INFO")

# # Perform download for each dataset. Data is already downloaded for the workshop
# for inst in mighti_insts:
#     inst.download(start_download_date, stop_download_date)
    
# # Change levels for logging printout to decrease feedback
# pysat.logger.setLevel("WARNING")

## Visualize with harkness
An easy way to visualize satellite data is to create a pointcloud with points at the locations expressed by the satellite coordinates, colormapped according to parameters of interest. This pointcloud can be visualized on a computer monitor with CloudCompare, or opened up in VR using VRifier or VR Point Cloud Editor.

### Generate pointclouds using harkness

In [ ]:
# ivm.load(date = start_download_date, end_date = stop_download_date)
ivm.load(2022, 15)
df = ivm.data
# Extract coordinates as separate lists
latitudes = df['Latitude'].tolist()
longitudes = df['Longitude'].tolist()
altitudes = df['Altitude'].tolist()
# Combine coordinates into a list of lists for sph2car
spherical_coords = list(zip(latitudes, longitudes, altitudes))
# Use sph2car to convert to Cartesian coordinates
cartesian_coords = coord.sph2car(spherical_coords)
# Unpack Cartesian coordinates into separate lists
x, y, z = zip(*cartesian_coords)  # * unpacks the results from sph2car
# Add new columns to DataFrame
df = df.assign(X=x, Y=y, Z=z)
print('Dataframe edited. Time to generate the .ply file...')

In [ ]:
df.head()
color_col = 'Ion_Density'
filename = 'output/'+color_col+'.ply'
create_ply(df, filename = filename, C=color_col, X='X', Y='Y', Z='Z', cmap='viridis', is_verbose=True)

### Visualize in Jupyter notebook

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
col = df[color_col]
ax.scatter(df['X'], df['Y'], df['Z'], c = col)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

## Create .3ds version (optional)

In [ ]:
# import aspose.threed as a3d

# scene = a3d.Scene.from_file("test.ply")
# scene.save("Output.3ds")